In [1]:
import pandas as pd
import math
import numpy as np
import itertools

In [2]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [3]:
stats = 'stats.csv'
games = 'games.csv'

In [25]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv(stats, header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('.','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('-','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].replace({'\$': '', '-': ''}, regex=True)

In [26]:
#The nba.com site changed its formatting so this is to remove non-existent rows

new_format_mask = 0
for index, row in statsDF.iterrows():
    if new_format_mask == 1:
        statsDF.drop(index, inplace=True)
        if row.iloc[0] == 'Totals:' and math.isnan(row.iloc[1]) == True:
            new_format_mask = 0 
        else:
            continue
    else:
        if row.iloc[0] == 'PLAYER' and row.iloc[1] != "MIN":
            new_format_mask += 1
        else:
            continue

for index, row in statsDF.iterrows():
    if row.iloc[0][3:4] == ":":
        statsDF.drop(index, inplace=True)

    

In [27]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.MIN != 'PLAYERS']
statsDF = statsDF[statsDF.PLAYER != 'Click']
statsDF = statsDF[statsDF.PLAYER != 'GLOSSARY']
statsDF = statsDF[statsDF.MIN != 'Filter']
statsDF = statsDF.reset_index(drop=True)
#Drop the first line for the team name
statsDF.drop(0, inplace=True)
statsDF = statsDF.reset_index(drop=True)

In [28]:
#This will delete the second time a team is named. 
for index, row in statsDF.iterrows():
    print(row.iloc[1])
    try: 
        if row.iloc[1].isalpha() and statsDF.iloc[index + 1][1].isalpha() or row.iloc[1] == "76ers":
            statsDF.drop(index, inplace=True)
            statsDF = statsDF.reset_index(drop=True)
            break
    except:
        continue

Ingram
3
Kuzma
7
Randle
13
Hart
9
Ennis
8
Brewer
4
Nance
2
Clarkson
2
Caruso
2
Bogut
3
Blue

CaldwellPope

Zubac

Rockets


In [29]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if row.iloc[0] == 'Nene':
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
    elif row.iloc[0].isalpha() == True and row.iloc[1].isalpha() == True:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Brandon Ingram', 'Kyle Kuzma', 'Julius Randle', 'Josh Hart', 'Tyler Ennis', 'Corey Brewer', 'Larry Nance', 'Jordan Clarkson', 'Alex Caruso', 'Andrew Bogut', 'Vander Blue', 'Kentavious CaldwellPope', 'Ivica Zubac', 'Trevor Ariza', 'Ryan Anderson', 'Clint Capela', 'James Harden', 'Chris Paul', 'PJ Tucker', 'Eric Gordon', 'Nene nan', 'Gerald Green', 'Tarik Black', 'Bobby Brown', 'Briante Weber', 'Zhou Qi']


In [30]:
#Setup dicts from the DF
statsDF_Dict = statsDF.T.to_dict().values()

In [31]:
statsDF

,PLAYER,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,OREB,DREB,REB,AST,TOV,STL,BLK,PF,PTS,+/-
0,Brandon,Ingram,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44:29,3,6,50.0,0,0,0.0,6,6,100,...,5,6,4,4,1,0,1,12,-6,NaN
2,Kyle,Kuzma,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42:51,7,20,35.0,5,10,50.0,4,4,100,...,4,5,3,1,0,2,4,23,-7,NaN
4,Julius,Randle,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,33:03,13,19,68.4,0,0,0.0,3,6,50.0,...,11,15,6,4,0,1,6,29,8,NaN
6,Josh,Hart,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,36:53,9,13,69.2,4,7,57.1,4,8,50.0,...,5,6,1,0,1,0,6,26,6,NaN
8,Tyler,Ennis,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,45:28,8,18,44.4,0,3,0.0,4,5,80.0,...,3,4,11,3,2,0,6,20,-3,NaN


In [32]:
dict_list = []
#Iterate through the statsDF and only return lines with stats or lines where players sat
for index, i in enumerate(statsDF_Dict):
    if index == 1 or index % 2 != 0:
        dict_list.append(i)
    

In [33]:
finalStats = pd.DataFrame(dict_list)
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')
finalStats['Player'] = finalStats.index

In [34]:
finalStats.head()

,FTM,3P%,3PA,TOV,PF,REB,3PM,FG%,FGA,OREB,...,FTA,FGM,DREB,PTS,MIN,+/-,AST,STL,BLK,Player
Brandon Ingram,6,0.0,0,4,1,6,0,50.0,6,1,...,6,3,5,12,44:29,-6,4,0,1,Brandon Ingram
Kyle Kuzma,4,50.0,10,1,4,5,5,35.0,20,1,...,4,7,4,23,42:51,-7,3,2,0,Kyle Kuzma
Julius Randle,3,0.0,0,4,6,15,0,68.4,19,4,...,6,13,11,29,33:03,8,6,1,0,Julius Randle
Josh Hart,4,57.1,7,0,6,6,4,69.2,13,1,...,8,9,5,26,36:53,6,1,0,1,Josh Hart
Tyler Ennis,4,0.0,3,3,6,4,0,44.4,18,1,...,5,8,3,20,45:28,-3,11,0,2,Tyler Ennis


In [35]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv(games, header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [36]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
0,LAL,HOU,HOUSTON,37,35,29,21,44,21,25,32,148,142,"Wood""""","Goble""""",Ford


In [37]:
for index,row in finalStats.iterrows():
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [38]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)
finalStats = finalStats.fillna('NA')

In [39]:
finalStats.index

Index(['Brandon Ingram', 'Kyle Kuzma', 'Julius Randle', 'Josh Hart',
       'Tyler Ennis', 'Corey Brewer', 'Larry Nance', 'Jordan Clarkson',
       'Alex Caruso', 'Andrew Bogut', 'Vander Blue', 'Kentavious CaldwellPope',
       'Ivica Zubac', 'Trevor Ariza', 'Ryan Anderson', 'Clint Capela',
       'James Harden', 'Chris Paul', 'PJ Tucker', 'Eric Gordon', 'Nene nan',
       'Gerald Green', 'Tarik Black', 'Bobby Brown', 'Briante Weber',
       'Zhou Qi'],
      dtype='object')